<a href="https://colab.research.google.com/github/DigiClau/stablediffusion_webui/blob/main/StableDiffusionWebUI_Forge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, subprocess, time, glob
#@title <font size="6" color="black">Stable Diffusion WebUI Forge</font>
output_path = 'Forge'
is_drive_used = True

DreamShaper = False #@param{type: "boolean"}
ReVAnimated = False #@param{type: "boolean"}
SDXL_10_Base_AND_Refiner = False #@param{type: "boolean"}

ControlNet = False #@param{type: "boolean"}
Deforum = False #@param{type: "boolean"}
SadTalker = False #@param{type: "boolean"}
Regional_Prompter = False #@param{type: "boolean"}
Ultimate_SD_Upscale = False #@param{type: "boolean"}
Openpose_Editor = False #@param{type: "boolean"}
ADetailer = False #@param{type: "boolean"}
AnimateDiff = False #@param{type: "boolean"}
Reactor = False #@param{type: "boolean"}
InpaintAnything = False #@param{type: "boolean"}
Replacer = False #@param{type: "boolean"}

LoopbackWave = False #@param{type: "boolean"}

# Depthmap_3D_Painting = True #@param{type: "boolean"}

# 로그 삭제
# Clear_Log = True #@param{type:'boolean'}

if is_drive_used:
  from google.colab import drive
  drive.mount('/content/drive')
  output_path = '/content/drive/MyDrive/' + output_path
  # Colab에 설치
  root = '/content/'
  sd_main = 'stable-diffusion-webui-forge'
  # 구글 드라이브에 필요한 폴터 생성
  !mkdir -p {output_path}/outputs
  !mkdir -p {output_path}/checkpoints
  !mkdir -p {output_path}/ESRGAN
  !mkdir -p {output_path}/hypernetworks
else:
  # Don't connect to google drive
  output_path = '/content/' + output_path # Is it still needed?
  root = '/content/'

def print_message(msg):
  # print('📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍')
  # print('📍📍📍 ', msg, ' 📍📍📍')
  # print('📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍📍')
  pass

def clear():
    from IPython.display import clear_output
    return clear_output()

def downloadModel(url):
  if 'huggingface.co' in url:
    filename = url.split('/')[-1]
    filename = filename.removesuffix('?download=true')
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}  -o {filename}
  elif 'civitai.com' in url: # CivitAP는 더이상 사용 안합니다.
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}
  else:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}
    print_message('지원 하지 않는 URL이지만, 다운로드를 시도했습니다. 확인해 보시기 바랍니다.')

def download_models():
  # Checkpoint 모델 다운로드
  %cd {root}/{sd_main}/models/Stable-diffusion
  if DreamShaper:
    downloadModel('https://huggingface.co/Lykon/DreamShaper/resolve/main/DreamShaper_8_pruned.safetensors')
  if ReVAnimated:
    downloadModel('https://civitai.com/api/download/models/46846')
  if SDXL_10_Base_AND_Refiner:
    downloadModel('https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors')
    downloadModel('https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors')

def install_extensions():
  # extensions directory
  %cd {root}/{sd_main}/extensions

  if ControlNet:
    !git clone https://github.com/Mikubill/sd-webui-controlnet.git
    %cd {root}/{sd_main}/extensions/sd-webui-controlnet/models
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime.pth')
    downloadModel('https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth')
    downloadModel('https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth')
    # brightness model
    downloadModel('https://huggingface.co/ioclab/ioc-controlnet/resolve/main/models/control_v1p_sd15_brightness.safetensors')
    # QR monster model
    downloadModel('https://huggingface.co/monster-labs/control_v1p_sd15_qrcode_monster/resolve/main/control_v1p_sd15_qrcode_monster.safetensors')
    downloadModel('https://huggingface.co/monster-labs/control_v1p_sd15_qrcode_monster/resolve/main/v2/control_v1p_sd15_qrcode_monster_v2.safetensors')
    downloadModel('https://huggingface.co/Nacholmo/controlnet-qr-pattern-sdxl/resolve/main/automatic1111/control_v01u_sdxl_qrpattern.safetensors')
    # SDXL model
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_canny_full.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_depth_full.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ip-adapter_xl.pth')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_blur.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_blur_anime.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_scribble_anime.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_recolor_256lora.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_sketch_256lora.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_diffusers_xl_lineart.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/thibaud_xl_openpose.safetensors')
    # insightface
    !pip install insightface
    %cd {root}/{sd_main}/extensions
  if Deforum:
    !git clone https://github.com/deforum-art/deforum-for-automatic1111-webui
    !apt-get install libvulkan1
  if SadTalker:
#    !git clone https://github.com/OpenTalker/SadTalker
#    !mkdir -p {root}/{sd_main}/models/SadTalker
#    %cd {root}/{sd_main}/models/SadTalker
#    url = 'https://huggingface.co/digiclau/files/resolve/main/extensions/SadTalker/sadtalker_checkpoints-20230517T015718Z.zip'
#    filename = "sadtalker_checkpoints.zip"
#    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url} -o {filename}
#    !unzip {filename}
#    !rm {filename}
#    # extra code to fix error
#    !mkdir /tmp
#    !mkdir /tmp/gradio
#    %cd {root}/{sd_main}/extensions
    !git clone https://github.com/OpenTalker/SadTalker
    !mkdir -p {root}/{sd_main}/models/SadTalker
    %cd {root}/{sd_main}/models/SadTalker
    url = 'https://huggingface.co/digiclau/files/resolve/main/extensions/SadTalker/sadtalker_checkpoints.zip'
    filename = "sadtalker_checkpoints.zip"
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url} -o {filename}
    !unzip {filename}
    !rm {filename}
#    # extra code to fix error
    !mkdir /tmp
    !mkdir /tmp/gradio
    %cd {root}/{sd_main}/extensions
  if Regional_Prompter:
    !git clone https://github.com/hako-mikan/sd-webui-regional-prompter
  if Ultimate_SD_Upscale:
    !git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111
  if Openpose_Editor:
    !git clone https://github.com/fkunn1326/openpose-editor
  if ADetailer:
    !git clone https://github.com/Bing-su/adetailer
  if AnimateDiff:
    !git clone https://github.com/continue-revolution/sd-webui-animatediff
    %cd {root}/{sd_main}/extensions/sd-webui-animatediff/model
    downloadModel('https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sd14_v1.safetensors')
    downloadModel('https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sd15_v1.safetensors')
    downloadModel('https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sd15_v2.safetensors')
    downloadModel('https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sd15_v3.safetensors')
    downloadModel('https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sdxl_hs.safetensors')
    downloadModel('https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sdxl_v10_beta.safetensors')

    downloadModel('https://huggingface.co/manshoety/AD_Stabilized_Motion/resolve/main/mm-Stabilized_high.pth')
    downloadModel('https://huggingface.co/manshoety/AD_Stabilized_Motion/resolve/main/mm-Stabilized_mid.pth')
    downloadModel('https://huggingface.co/manshoety/beta_testing_models/resolve/main/mm-p_0.5.pth')
    downloadModel('https://huggingface.co/manshoety/beta_testing_models/resolve/main/mm-p_0.75.pth')
    downloadModel('https://huggingface.co/CiaraRowles/TemporalDiff/resolve/main/temporaldiff-v1-animatediff.safetensors')

#    downloadModel('https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/lora/mm_sd15_v3_adapter.safetensors')
    %cd {root}/{sd_main}/extensions
  if Reactor:
    !git clone https://github.com/Gourieff/sd-webui-reactor.git

#  if Depthmap_3D_Painting:
#    !git clone https://github.com/thygate/stable-diffusion-webui-depthmap-script
  if InpaintAnything:
    !git clone https://github.com/Uminosachi/sd-webui-inpaint-anything
  if Replacer:
    !git clone https://github.com/continue-revolution/sd-webui-segment-anything
    !git clone https://github.com/light-and-ray/sd-webui-replacer
    %cd {root}/{sd_main}/extensions/sd-webui-segment-anything/models/sam
    downloadModel('https://huggingface.co/lkeab/hq-sam/resolve/main/sam_hq_vit_h.pth')
    %cd {root}/{sd_main}/extensions

def install_scripts():
  if LoopbackWave:
    %cd {root}/{sd_main}/scripts
#    url = 'https://files.catbox.moe/0hx51x.py'
#    filename = 'loopback_wave.py'
#    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}  -o {filename}
    url = 'https://github.com/DigiClau/stablediffusion_webui/raw/main/scripts/loopback_wave.py'
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}

def updatePython():
  !python --version > /content/pyversion
  with open('/content/pyversion', 'r') as file:
      if '3.10' in file.read():
        print_message('이미 python 3.10이 설치되어 있습니다.')
        return

  #install python 3.10.6
  !apt-get update -y
  !apt-get install python3.10.6

  #change alternatives
  !rm /usr/local/bin/python
  !rm /usr/local/bin/pip
  !sudo apt-get install python3.10-distutils
  !sudo update-alternatives --install /usr/local/bin/python python /usr/bin/python3.10 2
  !wget https://bootstrap.pypa.io/get-pip.py && python get-pip.py

def initSaveGoogleDriveModelOnly():
  # Use config file in google drive
  if not os.path.exists(output_path + '/ui-config.json'):
    !wget https://raw.githubusercontent.com/DigiClau/stablediffusion_webui/main/ui-config.json -O {output_path + '/ui-config.json'}
  if not os.path.exists(output_path + '/config.json'):
    !wget https://raw.githubusercontent.com/DigiClau/stablediffusion_webui/main/config.json -O {output_path + '/config.json'}
  if not os.path.exists(output_path + '/styles.csv'):
    print("Create new styles.csv file.")
    !wget https://raw.githubusercontent.com/DigiClau/stablediffusion_webui/main/style.csv -O {output_path + '/styles.csv'}

  !ln -s {output_path + '/ui-config.json'} {root}/{sd_main}/
  !ln -s {output_path + '/config.json'} {root}/{sd_main}/
  !ln -s {output_path + '/styles.csv'} {root}/{sd_main}/
  !ln -s {output_path}/outputs

  # embeddings folder on Google Drive
  !mkdir -p {output_path}/embeddings
  !rm -rf embeddings
  !ln -s {output_path}/embeddings

  # save parameter file in google drive
  if not os.path.exists(output_path + '/params.txt'):
    !touch {output_path + '/params.txt'}
  !ln -s {output_path}/params.txt

  # link all checkpoints in the checkpoints folder
  %cd {root}/{sd_main}/models/Stable-diffusion
  checkpoints_in_google_drive = glob.glob(output_path + '/checkpoints/*')
  print_message('구글 드라이브에 들어 있는 Checkpoint 모델: %s'%checkpoints_in_google_drive)
  for f in checkpoints_in_google_drive:
    !ln -s {f}

  # link all upscalers in the model folder
  !mkdir -p {root}/{sd_main}/models/ESRGAN
  %cd {root}/{sd_main}/models/ESRGAN
  upscalers_in_google_drive = glob.glob(output_path + '/ESRGAN/*')
  print_message('구글 드라이브에 들어 있는 Upscaler 모델: %s'%upscalers_in_google_drive)
  for f in upscalers_in_google_drive:
    !ln -s {f}

  # use lora model folder in google drive
  !mkdir -p {output_path}/Lora
  %cd {root}/{sd_main}/models
  !rm -rf Lora
  !ln -s {output_path}/Lora

  # use hypernetwork folder in google drive
  !mkdir -p {output_path}/hypernetworks
  %cd {root}/{sd_main}/models
  !rm -rf hypernetworks
  !ln -s {output_path}/hypernetworks

  # use VAE folder in google rive
  !mkdir -p {output_path}/VAE
  %cd {root}/{sd_main}/models
  !rm -rf VAE
  !ln -s {output_path}/VAE

  # use ControlNet folder in google rive
  !mkdir -p {output_path}/ControlNet
  %cd {root}/{sd_main}/models
  !rm -rf ControlNet
  !ln -s {output_path}/ControlNet

updatePython()
!mkdir -p {root}
%cd {root}
# os.chdir(root)
!apt-get -y install -qq aria2
!pip install pyngrok
!git clone https://github.com/lllyasviel/stable-diffusion-webui-forge
# A1111 first launch
%cd {root}/{sd_main}

# !git checkout -f f11456e # latest
!COMMANDLINE_ARGS="--exit"  python launch.py

%cd {root}/{sd_main}
args = f'--gradio-img2img-tool color-sketch --enable-insecure-extension-access --gradio-queue'
args += ' --share '
!python {root}/{sd_main}/launch.py {args}
